In [5]:
from datasets import load_dataset

dataset_path = "Qwen2-Magpie-Pro-1M-v0.1.jsonl"
dataset = load_dataset("json", data_files=dataset_path)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['uuid', 'model', 'gen_input_configs', 'instruction', 'response', 'conversations', 'task_category', 'other_task_category', 'task_category_generator', 'difficulty', 'intent', 'knowledge', 'difficulty_generator', 'input_quality', 'quality_explanation', 'quality_generator', 'llama_guard_2', 'reward_model', 'instruct_reward', 'min_neighbor_distance', 'repeat_count', 'min_similar_uuid', 'instruction_length', 'response_length', 'language'],
        num_rows: 1000000
    })
})


In [6]:
def high_quality_filter(example):
    return (
        example['input_quality'] in ['good', 'excellent']
        and example['instruct_reward'] > -10
        and not example['instruction'].endswith(':')
        and (
            example['min_similar_uuid'] is None
            or example['uuid'] == example['min_similar_uuid']
        )
    )
filtered_dataset = dataset['train'].filter(high_quality_filter)
print(filtered_dataset)

Dataset({
    features: ['uuid', 'model', 'gen_input_configs', 'instruction', 'response', 'conversations', 'task_category', 'other_task_category', 'task_category_generator', 'difficulty', 'intent', 'knowledge', 'difficulty_generator', 'input_quality', 'quality_explanation', 'quality_generator', 'llama_guard_2', 'reward_model', 'instruct_reward', 'min_neighbor_distance', 'repeat_count', 'min_similar_uuid', 'instruction_length', 'response_length', 'language'],
    num_rows: 400517
})


In [3]:
def get_output_length(example):
    return len(example['response'])

filtered_dataset = filtered_dataset.map(lambda x: {'response_length': get_output_length(x)})
# sort the dataset by response_length
sorted_dataset = filtered_dataset.sort('response_length', reverse=True)
# select the top 300000 examples
top_x_dataset = sorted_dataset.select(range(300000))
# Shuffle the dataset
top_x_dataset = top_x_dataset.shuffle(seed=42)

Map:   0%|          | 0/400517 [00:00<?, ? examples/s]

In [4]:
# Submit the dataset to the hub
top_x_dataset.push_to_hub("Magpie-Align/Magpie-Qwen-Pro-300K-Filtered", private=True)

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Magpie-Align/Magpie-Qwen-Pro-300K-Filtered/commit/fe26468dee24771971ae2ed93875c5a5219ed746', commit_message='Upload dataset', commit_description='', oid='fe26468dee24771971ae2ed93875c5a5219ed746', pr_url=None, pr_revision=None, pr_num=None)